In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from stable_baselines3 import PPO
from PIDController import PIDController  # Your PID controller class
from OT2Eenv import OT2Env  # Custom PID environment
from ot2_env_wrapper import OT2Env  # Custom RL environment

# Paths to RL models
model_paths = [
    r"C:\Users\Edopi\Downloads\model (7).zip",
]

# Fixed goal position
fixed_goal_position = np.array([0.1, 0.1, 0.2], dtype=np.float32)


def evaluate_rl_model(model_path, env, fixed_goal, num_episodes=10):
    """
    Evaluate an RL model on its environment and return performance metrics.
    """
    model = PPO.load(model_path)
    lowest_error = float('inf')
    total_steps = 0
    cumulative_error = 0

    for episode in range(num_episodes):
        obs, _ = env.reset()
        env.goal_position = fixed_goal
        done = False
        episode_steps = 0

        while not done:
            current_position = obs[:3]
            distance_to_goal = np.linalg.norm(current_position - fixed_goal)
            cumulative_error += distance_to_goal
            episode_steps += 1
            lowest_error = min(lowest_error, distance_to_goal)

            action, _ = model.predict(obs)
            obs, reward, done, truncated, info = env.step(action)

            if truncated:
                break

        total_steps += episode_steps

    avg_error = cumulative_error / (num_episodes * total_steps)
    return lowest_error, total_steps / num_episodes, avg_error


def evaluate_pid_controller(env, fixed_goal):
    """
    Evaluate the PID controller on its environment and return performance metrics.
    """
    # PID controllers for X, Y, Z axes
    pid_x = PIDController(kp=10, ki=5, kd=0.01)
    pid_y = PIDController(kp=10, ki=5, kd=0.01)
    pid_z = PIDController(kp=10, ki=5, kd=0.01)

    obs, _ = env.reset()
    env.goal_position = fixed_goal
    done = False

    total_steps = 0
    cumulative_error = 0
    lowest_error = float('inf')

    while not done:
        current_position = obs[:3]
        error_x = fixed_goal[0] - current_position[0]
        error_y = fixed_goal[1] - current_position[1]
        error_z = fixed_goal[2] - current_position[2]

        control_x = pid_x.compute(error_x)
        control_y = pid_y.compute(error_y)
        control_z = pid_z.compute(error_z)

        action = np.array([control_x, control_y, control_z], dtype=np.float32)

        obs, reward, done, truncated, info = env.step(action)
        distance_to_goal = np.linalg.norm(current_position - fixed_goal)
        cumulative_error += distance_to_goal
        total_steps += 1
        lowest_error = min(lowest_error, distance_to_goal)

        if truncated:
            break

    avg_error = cumulative_error / total_steps
    return lowest_error, total_steps, avg_error


def main():
    """
    Main function to compare RL and PID controllers.
    """
    # Initialize separate environments for RL and PID
    env_rl = OT2Env(render=False, max_steps=2000)
    env_pid = OT2Env(render=False, max_steps=2000)

    results = {}

    try:
        # Evaluate RL models
        for model_path in model_paths:
            model_name = model_path.split("\\")[-1]  # Extract model name
            print(f"Evaluating RL model: {model_name}")
            lowest_error, avg_steps, avg_error = evaluate_rl_model(model_path, env_rl, fixed_goal_position)
            results[model_name] = {
                "Lowest Error": lowest_error,
                "Avg Steps": avg_steps,
                "Avg Error": avg_error
            }
            print(f"RL Model: {model_name} | Lowest Error: {lowest_error:.6f} m | "
                  f"Avg Steps: {avg_steps:.2f} | Avg Error: {avg_error:.6f} m")

        # Evaluate PID controller
        print("\nEvaluating PID controller...")
        lowest_error, total_steps, avg_error = evaluate_pid_controller(env_pid, fixed_goal_position)
        results["PID"] = {
            "Lowest Error": lowest_error,
            "Avg Steps": total_steps,
            "Avg Error": avg_error
        }
        print(f"PID Controller | Lowest Error: {lowest_error:.6f} m | "
              f"Total Steps: {total_steps} | Avg Error: {avg_error:.6f} m")

        # Compare results
        print("\nPerformance Comparison:")
        for model, metrics in results.items():
            print(f"{model}: {metrics}")

        # Visualize comparison
        labels = list(results.keys())
        lowest_errors = [metrics["Lowest Error"] for metrics in results.values()]
        avg_steps = [metrics["Avg Steps"] for metrics in results.values()]
        avg_errors = [metrics["Avg Error"] for metrics in results.values()]

        x = np.arange(len(labels))
        width = 0.3

        plt.figure(figsize=(12, 6))

        plt.bar(x - width, lowest_errors, width, label='Lowest Error')
        plt.bar(x, avg_steps, width, label='Average Steps')
        plt.bar(x + width, avg_errors, width, label='Average Error')

        plt.xticks(x, labels)
        plt.ylabel("Metric Value")
        plt.title("RL vs PID Controller Performance Comparison")
        plt.legend()
        plt.grid()
        plt.tight_layout()
        plt.show()

    finally:
        # Ensure environments are properly closed
        print("Closing environments...")
        if env_rl:
            env_rl.close()
        if env_pid:
            env_pid.close()


if __name__ == "__main__":
    main()


Evaluating RL model: model (7).zip
Reset: Pipette Position [0.073  0.0895 0.1195], Goal Position [ 0.07771875 -0.02294771  0.19206953]
Step 1 called with action: [ 0.07503133 -1.          1.        ]
Step 2 called with action: [0.8972838  0.33796686 1.        ]
Step 3 called with action: [0.81821984 0.45269215 1.        ]
Step 4 called with action: [ 0.6068134  -0.12489125  1.        ]
Step 5 called with action: [0.77803814 0.15842059 1.        ]
Step 6 called with action: [0.8925065  0.65281177 1.        ]
Step 7 called with action: [0.09647667 0.75963384 1.        ]
Step 8 called with action: [1.        0.5467085 1.       ]
Step 9 called with action: [0.6182363  0.70565313 1.        ]
Step 10 called with action: [0.89956445 0.1067746  1.        ]
Step 11 called with action: [0.2285657 0.2215192 1.       ]
Step 12 called with action: [0.29805672 0.2328752  1.        ]
Step 13 called with action: [0.83724546 0.6140191  1.        ]
Step 14 called with action: [0.9437152  0.42811084 1.  